In [2]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer ## pytorch trainer sınıfını kullanır

In [5]:
dataset = load_dataset("rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [6]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
##veri setindeki sadece text kısmına tokenizer uygulucam
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"])

dataset = dataset.map(tokenize_dataset, batched=True)
"""
Kodunuzda dataset.map() fonksiyonu, tokenize_dataset adlı bir fonksiyonu dataset veri kümesine uyguluyor.
 Bu işlem sırasında her bir örneği tek tek değil, gruplar (batch'ler) halinde işlemeye olanak tanıyor.
  Bu şekilde işlem, hız ve verimlilik açısından optimize edilmiş oluyor.
"""

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
"""
DataCollatorWithPadding, her batch içindeki dizilerin (token id, attention mask gibi özellikler) aynı uzunlukta olmasını sağlar.
Çünkü bir batch'teki her örnek, modelin girişine girebilmesi için aynı uzunlukta olmalıdır.
Metin verilerinde her cümlenin uzunluğu farklı olabileceğinden,
kısa cümleleri daha uzun olanlarla eşit uzunluğa getirmek için padding (doldurma) yapılır.
tokenizer=tokenizer: Bu parametre, DataCollatorWithPadding nesnesine hangi tokenizer'ın kullanılacağını söyler.
"""

"\nDataCollatorWithPadding, her batch içindeki dizilerin (token id, attention mask gibi özellikler) aynı uzunlukta olmasını sağlar.\nÇünkü bir batch'teki her örnek, modelin girişine girebilmesi için aynı uzunlukta olmalıdır. \nMetin verilerinde her cümlenin uzunluğu farklı olabileceğinden,\nkısa cümleleri daha uzun olanlarla eşit uzunluğa getirmek için padding (doldurma) yapılır.\ntokenizer=tokenizer: Bu parametre, DataCollatorWithPadding nesnesine hangi tokenizer'ın kullanılacağını söyler.\n"

In [17]:
training_args=TrainingArguments(
    output_dir = "my_bert_model",
    learning_rate = 2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    report_to = "none"
)
"""
output_dir="my_bert_model": Eğitilmiş modelin kaydedileceği dizini belirtir.
learning_rate=2e-5: Öğrenme oranını belirler. Bu değer, modelin ağırlıklarının her adımda ne kadar değişeceğini kontrol eder.
per_device_train_batch_size=8 ve per_device_eval_batch_size=8: Eğitim ve değerlendirme sırasında her cihaz (örneğin GPU) için batch boyutunu belirtir.
num_train_epochs=2: Eğitim döngüsü (epoch) sayısını belirler. Model tüm veri setini bu sayı kadar görecektir.
report_to="none": Eğitim sürecindeki ilerlemelerin raporlanmasıyla ilgili ayarlardır. "none" olarak ayarlanmış, yani eğitim sırasında izleme yapılmayacak.

"""

'\noutput_dir="my_bert_model": Eğitilmiş modelin kaydedileceği dizini belirtir.\nlearning_rate=2e-5: Öğrenme oranını belirler. Bu değer, modelin ağırlıklarının her adımda ne kadar değişeceğini kontrol eder.\nper_device_train_batch_size=8 ve per_device_eval_batch_size=8: Eğitim ve değerlendirme sırasında her cihaz (örneğin GPU) için batch boyutunu belirtir.\nnum_train_epochs=2: Eğitim döngüsü (epoch) sayısını belirler. Model tüm veri setini bu sayı kadar görecektir.\nreport_to="none": Eğitim sürecindeki ilerlemelerin raporlanmasıyla ilgili ayarlardır. "none" olarak ayarlanmış, yani eğitim sırasında izleme yapılmayacak.\n\n'

In [18]:
## pytorch eğitim için trainer sınıfını kullanır
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator
)
"""
Trainer, Hugging Face transformers kütüphanesindeki temel eğitim sınıfıdır ve modelin eğitim ve değerlendirme adımlarını basitleştirir.
Trainer nesnesi oluşturulurken, model, eğitim parametreleri, veri kümeleri ve tokenizer gibi eğitim için gerekli tüm bileşenler tanımlanır.
"""

'\nTrainer, Hugging Face transformers kütüphanesindeki temel eğitim sınıfıdır ve modelin eğitim ve değerlendirme adımlarını basitleştirir.\nTrainer nesnesi oluşturulurken, model, eğitim parametreleri, veri kümeleri ve tokenizer gibi eğitim için gerekli tüm bileşenler tanımlanır.\n'

In [19]:
trainer.train()

Step,Training Loss


Step,Training Loss
500,0.364500
1000,0.382400
1500,0.244100
2000,0.252700


TrainOutput(global_step=2134, training_loss=0.305343138132569, metrics={'train_runtime': 5910.2152, 'train_samples_per_second': 2.887, 'train_steps_per_second': 0.361, 'total_flos': 195974132394480.0, 'train_loss': 0.305343138132569, 'epoch': 2.0})

In [ ]:
text = "I love NLP. It's fun to analyze the NLP tasks with HuggingFace"

In [20]:
inputs = tokenizer(text, return_tensors = "pt")
inputs

NameError: name 'text' is not defined

In [ ]:
model_path="/kaggle/working/my_bert_model/checkpoint-1000"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=2
)

In [ ]:
import torch

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits
"""
 tahmin veya değerlendirme sırasında kullanılmak üzere optimize edilmiş bir işlemi temsil eder.
 Burada torch.no_grad() yapısı, modelin tahminleri sırasında gradyan hesaplamalarını devre dışı bırakır,
 bu da bellek tasarrufu ve hız açısından avantaj sağlar.

torch.no_grad(), PyTorch’ta gradyanların hesaplanmasını engelleyen bir yapı sağlar.
 Modelin yalnızca tahmin yaptığı durumlarda kullanılır; çünkü gradyan hesaplamasına gerek yoktur.
Avantajları:
Bellek Tasarrufu: Gradyan hesaplanmadığı için daha az bellek kullanılır.
Hız Artışı: Özellikle büyük veri kümelerinde işlem hızı artar.
Bu yapı, model tahmini sırasında bellekteki gereksiz yükü azaltır ve işlemi hızlandırır.

logits = model(**inputs).logits
modelin ileri geçiş (forward pass) yaparak girdi verisinden logits adlı ham çıktı değerlerini elde etmesini sağlar.
model(**inputs): inputs sözlük yapısında olduğu için ** kullanılarak modelin ileri geçişine bu girdi verilir.
Bu yaklaşım, modelin yalnızca çıkarım (inference) veya değerlendirme işlemlerinde kullanılması gereken hızlı ve hafif bir yöntemdir.
    """

In [ ]:
logits.argmax().item()

1